## Before coding

Create a new project

https://console.developers.google.com/projectcreate

Once you have created the project, enable access to the YouTube Data API

https://console.developers.google.com/apis/library

Once enabled, it is important that you get credentials for your project

https://console.developers.google.com/apis/credentials/wizard?api=youtube.googleapis.com

From the options select:  

| Option  | Value |
| ------------- | ------------- |
| ¿Qué API estás usando?  | **YouTube Data API v3**  |
| ¿Desde dónde llamarás a la API? | **Servidor Web**  |
| ¿A qué tipo de datos accederás? | **Datos públicos**  |  

Having selected such values, press: **"¿Qué credenciales necesito?"**  and you will be given an alphanumeric string that is your API key, place this value into the `api_key` variable:

In [ ]:
api_key = ""

## Now, coding

Import the necessary packages

In [ ]:
import requests
import json
import urllib
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from slugify import slugify

In [ ]:
search_url = "https://www.googleapis.com/youtube/v3/search?"
caption_url = "https://www.youtube.com/api/timedtext?"

In [ ]:
language_preferences = ['es-MX','es']
channels = {
    'presidencia' : 'UCfBRIj1Tq8k7-SD8PPVPJpQ',
    'epn': 'UC4hlUPhJxEvAwW-yli8kHpw',
    'amlo': 'UCxEgOKuI-n-WOJaNcisHvSg',
}

In [ ]:
parameters = {
    'key': api_key,
    'part': 'snippet',
    'type': 'video',
    'channelId': 'UCfBRIj1Tq8k7-SD8PPVPJpQ',
    'maxResults': 10,
    'order': 'relevance',
    'q': 'discurso'
}
query_string = urlencode(parameters)
print(urlencode(parameters))

In [ ]:
videos = {}
count = 0
for channel in channels:
    print("Searching for", channel)
    parameters['channelId'] = channels[channel]
    videos[channel] = []
    pages = 1
    if 'pageToken' in parameters:
        del(parameters['pageToken'])
        query_string = urlencode(parameters)
    page_token = 'FIRST TIME!'
    while pages > 0 and len(page_token) > 0:
#        print(query_string)
        qurl = search_url + query_string
        print(qurl)
        r = requests.get(search_url + query_string)
        result = json.loads(r.text)
        try:
            page_token = result["nextPageToken"]
        except:
            page_token = ''
        parameters['pageToken'] = page_token
        pages = pages - 1
        print(len(result['items']))
        videos[channel].extend(result['items'])
        count += len(result['items'])
    
print("Finished, found", count)

In [ ]:
vids_subs = {}
for channel in channels:
    vids_subs[channel] = []
    for video in videos[channel]:
        videoId = video['id']['videoId']
        title = video['snippet']['title']
        subtitles = ''
        i = 0
        yt = YouTube('https://www.youtube.com/watch?v=' + videoId)
        while len(subtitles) == 0 and i < len(language_preferences):
            lang = language_preferences[i]
            if yt.captions.get_by_language_code(lang) is not None:
                subtitles = yt.captions.get_by_language_code(lang).xml_captions
            i = i + 1
        if len(subtitles) > 0:
            vids_subs[channel].append({'id': videoId, 'title': title, 'captions': subtitles })
    print('Done!')

In [ ]:
for channel in channels:
    for subs in vids_subs[channel]:
        soup = BeautifulSoup(subs['captions'], "lxml")
        texts = soup.find_all('text')
        sub_entries = []
        for text in texts:
            sub_entry = {
                'duration': text.get('dur'),
                'start': text.get('start'),
                'content': text.string
            }
            sub_entries.append(sub_entry)
        del(subs['captions'])
        subs['captions_parsed'] = sub_entries

In [ ]:
# Save all the good stuff:
import os
from os.path import join


base_dir = 'youtube-captions'
for channel in channels:
    directory = join(base_dir, slugify(channel))
    if not os.path.exists(directory):
        os.makedirs(directory)

    print("Saving to", directory)
    for vid in vids_subs[channel]:
        file_path = join(directory, slugify(vid['title']) + '.json')
        with open(file_path, 'w') as outfile:
            json.dump(vid, outfile)
print("Done!")